In [398]:
import pandas as pd
import numpy as np
import folium
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup as BS
import re
import warnings
import bokeh
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import codecs
from json import JSONDecoder
from functools import partial

from folium import plugins
from folium.plugins import HeatMap
import datetime
import geopy.distance
#import mpu
from folium.plugins import MarkerCluster


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score


import numpy as np
from sklearn.preprocessing import Imputer

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
warnings.filterwarnings("ignore")

In [399]:
# !pip install --no-cache numpy==1.16.2
# !pip install folium
#!pip install geopy

In [400]:
faults = pd.read_csv('data\\J1939Faults.csv',header=None)


In [401]:
columns1=["FaultId",
"ESS_Id",
"EventTimeStamp",
"eventDescription",
"actionDescription",
"ecuSoftwareVersion",
"ecuSerialNumber",
"ecuModel",
"ecuMake",
"ecuSource",
"spn",
"fmi",
"active",
"activeTransitionCount",
"faultValue",
"EquipmentID",
"MCTNumber",
"Latitude",
"Longitude",
"LocationTimeStamp"] 

In [402]:
faults.columns=columns1
faults.shape

(1085683, 20)

In [403]:

faults.shape

(1085683, 20)

In [404]:
faults.EventTimeStamp = pd.to_datetime(faults.EventTimeStamp, format = '%Y-%m-%d %H:%M:%S')

In [405]:
faults['day'] = faults['EventTimeStamp'].dt.day
faults['month'] = faults['EventTimeStamp'].dt.month
faults['Year'] = faults['EventTimeStamp'].dt.year

In [406]:
faults["winter"]=0
faults.loc[faults['month'].isin([12,1,2]),"winter"]=1

In [407]:
pd.value_counts(faults['winter'])

0    812300
1    273383
Name: winter, dtype: int64

In [408]:
#faults = faults.loc[faults['active'] == True,:]

In [409]:
faults['EquipmentID'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
faults.shape

(1085683, 24)

In [410]:
faults['EquipmentID']=faults['EquipmentID'].astype(int)

In [411]:
faults['EquipmentID'].isna().sum()

0

In [412]:
faults=faults.loc[~faults['EquipmentID'].isin([105349576,105360462,105411041,105427203]), :]
faults.shape

(1085470, 24)

In [413]:
faults.head()

,FaultId,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,day,month,Year,winter
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000,21,2,2015,1
1,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000,21,2,2015,1
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000,21,2,2015,1
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000,21,2,2015,1
4,5,990416,2015-02-21 11:39:41,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000,21,2,2015,1


In [414]:
faults.loc[faults['ecuMake'].isin(['PCAR', '????R',
                                 '?CAR', '?ACCR', '???R']), 'ecuMake']="PACCR"
faults.loc[faults['ecuMake'].isin(['?MMNS', '??MNS']), 'ecuMake']="CMMNS"
faults.loc[faults['ecuMake']=='?NDWS', 'ecuMake']="BNDWS"

In [415]:
faults=faults.loc[(faults["ecuMake"]=="CMMNS"),:]
#faults=faults.loc[(faults["EquipmentID"]>9999),:]
#faults["EquipmentID"].astype(int)

In [416]:
faults.shape

(409759, 24)

In [417]:
bk=faults.groupby("FaultId").agg({"FaultId": [np.size]})
bk.head()

,FaultId
,size
FaultId,
6,1
8,1
9,1
10,1
11,1


In [418]:
list(bk)

[('FaultId', 'size')]

In [419]:
#bk["FaultId"].sort_values("size").head()

In [420]:
diagnostic = pd.read_csv('data\\VehicleDiagnosticOnboardData.csv',header=None)

In [421]:
diagnostic.head()

,0,1,2,3
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [422]:
columns2=["Id","Name","Value","FaultId"]

diagnostic.columns=columns2

diagnostic["Value"].shape

(11687087,)

In [423]:
diagnostic.head()

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [424]:
#merged.isna().sum()

In [425]:
merged=faults.set_index('FaultId').join(diagnostic.pivot(
  index = 'FaultId', 
  columns = 'Name', 
  values = 'Value')).reset_index().set_index('FaultId')

In [426]:
bl=diagnostic.pivot(
  index = 'FaultId', 
  columns = 'Name', 
  values = 'Value')

In [427]:
bl.head()

Name,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultId,,,,,,,,,,,,,,,,,,,,,,,,
1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN


In [428]:
merged.shape

(409759, 47)

In [429]:
merged.head()

,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,day,month,Year,winter,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,990431,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000,21,2,2015,1,48,14.4275,False,64.6226,470381.4,181.4,30,38.28,196.5313,1514.5,9480,44,70349.809963756,4.583399,NaN,True,111.2,1023,NaN,NaN,13.6022,3276.75,NaN,6.67
8,990441,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1429,105356054,38.228796,-84.582500,2015-02-21 11:41:44.000,21,2,2015,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
9,990442,2015-02-21 11:40:22,High (Severity Low) Water In Fuel Indicator,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,97,15,True,1,NaN,1429,105356054,38.228796,-84.582500,2015-02-21 11:41:44.000,21,2,2015,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
10,990446,2015-02-21 11:41:55,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1417,105438630,33.039953,-96.182592,2015-02-21 11:41:51.000,21,2,2015,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
11,990459,2015-02-21 11:42:19,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1582,105410980,40.158055,-80.151018,2015-02-21 11:42:55.000,21,2,2015,1,NaN,14.1375,False,64.6226,257885.9,181.4,80,34.22,218.975,1382.375,5085.35,42.8,37724.957799834,13.72378,NaN,True,78.8,1023,NaN,NaN,64.82649,3276.75,NaN,17.69


In [430]:
merged["full_derated"]=0
merged["partial_derated"]=0

merged.loc[(((merged["spn"]==5246) & (merged["fmi"]==0)) | ((merged["spn"]==1569) & (merged["fmi"]==31))),"full_derated"]=1

merged.loc[((merged["spn"]==1569) & (merged["fmi"]==31)),"partial_derated"]=1

In [431]:
merged['fmi_spn'] = "c_"+merged.fmi.map(str) + "_" + merged.spn.map(str)
faul_combined=merged[["fmi_spn"]]
faul_combined["value"]=1

In [432]:
data_reshape=faul_combined.pivot(columns='fmi_spn', values='value').reset_index()
data_reshape.head()
data_reshape.fillna(0)
data_reshape.head()

fmi_spn,FaultId,c_0_101,c_0_110,c_0_125,c_0_157,c_0_175,c_0_190,c_0_32000,c_0_3242,c_0_3246,c_0_3251,c_0_4360,c_0_4363,c_0_5246,c_0_5862,c_10_102,c_10_1761,c_10_3216,c_10_3226,c_10_3251,c_10_3364,c_10_3515,c_10_4360,c_10_4363,c_10_5024,c_10_5031,c_10_5853,c_10_84,c_11_1322,c_11_1326,c_11_1328,c_11_1761,c_11_190,c_11_3360,c_11_3364,c_11_5743,c_11_641,c_12_3226,c_12_3360,c_12_3361,c_12_5742,c_12_5743,c_12_5848,c_12_629,c_12_630,c_12_641,c_13_237,c_13_2791,c_13_3360,c_13_5848,c_13_639,c_13_641,c_13_6713,c_14_3936,c_14_4792,c_14_521032,c_14_641,c_15_101,c_15_102,c_15_157,c_15_3242,c_15_3246,c_15_3720,c_15_3936,c_15_95,c_15_97,c_16_101,c_16_102,c_16_1209,c_16_1239,c_16_157,c_16_168,c_16_175,c_16_188,c_16_2659,c_16_3058,c_16_3216,c_16_3242,c_16_3246,c_16_3251,c_16_3327,c_16_3363,c_16_3936,c_16_412,c_16_4331,c_16_4334,c_16_4360,c_16_4363,c_16_444,c_16_4765,c_16_5246,c_16_5742,c_16_5862,c_16_6773,c_16_81,c_16_95,c_16_97,c_17_102,c_17_111,c_17_168,c_17_1761,c_17_3480,c_17_5298,c_17_5394,c_17_65535,c_18_100,c_18_102,c_18_103,c_18_105,c_18_110,c_18_111,c_18_1127,c_18_1176,c_18_157,c_18_167,c_18_168,c_18_1761,c_18_188,c_18_3031,c_18_3058,c_18_3060,c_18_3364,c_18_3521,c_18_3556,c_18_3597,c_18_4094,c_18_4331,c_18_4334,c_18_4364,c_18_444,c_18_4607,c_18_5298,c_18_5585,c_18_5851,c_18_65535,c_19_3360,c_19_84,c_19_91,c_1_100,c_1_111,c_1_1176,c_1_1761,c_1_3556,c_1_4334,c_20_102,c_20_3216,c_20_3226,c_20_65535,c_21_3226,c_21_5835,c_29_196608,c_2_100,c_2_101,c_2_102,c_2_105,c_2_110,c_2_1172,c_2_1209,c_2_1279,c_2_171,c_2_175,c_2_190,c_2_255,c_2_2629,c_2_27,c_2_3031,c_2_3216,c_2_3218,c_2_3226,c_2_3228,c_2_3241,c_2_3245,c_2_3246,c_2_3251,c_2_3360,c_2_3361,c_2_3480,c_2_3482,c_2_3556,c_2_3610,c_2_3695,c_2_411,c_2_4334,c_2_4363,c_2_4375,c_2_4765,c_2_5019,c_2_51,c_2_5569,c_2_5745,c_2_5851,c_2_612,c_2_65535,c_2_723,c_2_7854,c_2_84,c_2_91,c_31_102,c_31_110,c_31_1322,c_31_1323,c_31_1324,c_31_1325,c_31_1326,c_31_1327,c_31_1328,c_31_1569,c_31_3362,c_31_3703,c_31_4094,c_31_4096,c_31_4364,c_31_4794,c_31_4796,c_31_524287,c_31_5319,c_31_5321,c_31_5397,c_31_633,c_31_641,c_31_65535,c_31_6802,c_31_7827,c_3_100,c_3_101,c_3_102,c_3_105,c_3_1072,c_3_1073,c_3_1075,c_3_108,c_3_110,c_3_111,c_3_1172,c_3_1209,c_3_1347,c_3_1349,c_3_157,c_3_171,c_3_175,c_3_1761,c_3_2623,c_3_3031,c_3_3242,c_3_3246,c_3_3251,c_3_3361,c_3_3363,c_3_3364,c_3_3464,c_3_3480,c_3_3482,c_3_3490,c_3_3513,c_3_3610,c_3_411,c_3_412,c_3_4334,c_3_4340,c_3_4342,c_3_4344,c_3_4360,c_3_4363,c_3_4376,c_3_4765,c_3_4813,c_3_5019,c_3_5097,c_3_51,c_3_5394,c_3_5491,c_3_5742,c_3_5743,c_3_5835,c_3_5862,c_3_647,c_3_65535,c_3_677,c_3_6780,c_3_729,c_3_7854,c_3_862,c_3_96,c_3_97,c_4_100,c_4_101,c_4_102,c_4_108,c_4_110,c_4_111,c_4_1172,c_4_1209,c_4_1347,c_4_171,c_4_1761,c_4_2623,c_4_27,c_4_3031,c_4_3216,c_4_3226,c_4_3242,c_4_3245,c_4_3246,c_4_3251,c_4_3361,c_4_3363,c_4_3480,c_4_3490,c_4_3509,c_4_3510,c_4_3511,c_4_3512,c_4_3513,c_4_3583,c_4_3610,c_4_411,c_4_4334,c_4_4340,c_4_4342,c_4_4360,c_4_4363,c_4_4375,c_4_4376,c_4_4765,c_4_5019,c_4_520953,c_4_5394,c_4_5491,c_4_5742,c_4_5743,c_4_5745,c_4_5746,c_4_5835,c_4_5848,c_4_5862,c_4_7321,c_4_7854,c_4_91,c_4_97,c_5_2791,c_5_3361,c_5_3363,c_5_3556,c_5_4340,c_5_4342,c_5_4344,c_5_4346,c_5_4376,c_5_4607,c_5_5394,c_5_5491,c_5_651,c_5_652,c_5_653,c_5_654,c_5_655,c_5_65535,c_5_656,c_6_2791,c_7_103,c_7_3362,c_7_3363,c_7_3464,c_7_3482,c_7_3490,c_7_3936,c_7_4339,c_7_4376,c_7_4752,c_7_5394,c_7_5491,c_7_641,c_7_651,c_7_652,c_7_653,c_7_654,c_7_655,c_7_656,c_7_723,c_8_2623,c_8_91,c_9_1081,c_9_1761,c_9_191,c_9_3031,c_9_3216,c_9_3226,c_9_3360,c_9_3364,c_9_38,c_9_558,c_9_563,c_9_5742,c_9_5743,c_9_5835,c_9_5848,c_9_639,c_9_641,c_9_65535,c_9_6713,c_9_84,c_9_91,c_9_96
0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [433]:
merged=pd.merge(merged,data_reshape,on="FaultId",how="outer")

In [434]:
pd.value_counts(merged.full_derated).to_frame().reset_index()

,index,full_derated
0,0,400562
1,1,9197


In [435]:
merged["BarometricPressure"] = pd.to_numeric(merged["BarometricPressure"])


In [436]:
merged["CruiseControlSetSpeed"] = pd.to_numeric(merged["CruiseControlSetSpeed"])

In [437]:
merged["DistanceLtd"] = pd.to_numeric(merged["DistanceLtd"])
merged["EngineCoolantTemperature"] = pd.to_numeric(merged["EngineCoolantTemperature"])

In [438]:
merged["EngineLoad"] = pd.to_numeric(merged["EngineLoad"])
merged["EngineOilPressure"] = pd.to_numeric(merged["EngineOilPressure"])
merged["EngineOilTemperature"] = pd.to_numeric(merged["EngineOilTemperature"])

In [439]:
merged["EngineRpm"] = pd.to_numeric(merged["EngineRpm"])
merged["EngineTimeLtd"] = pd.to_numeric(merged["EngineTimeLtd"])

In [440]:
merged["TurboBoostPressure"] = pd.to_numeric(merged["TurboBoostPressure"])
merged["Speed"] = pd.to_numeric(merged["Speed"])

In [441]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 409759 entries, 0 to 409758
Columns: 453 entries, FaultId to c_9_96
dtypes: bool(1), datetime64[ns](1), float64(417), int32(1), int64(13), object(20)
memory usage: 1.4+ GB


In [442]:
regression_data=merged

In [443]:
regression_data.shape

(409759, 453)

In [444]:
pd.value_counts(regression_data.full_derated).to_frame().reset_index()

,index,full_derated
0,0,400562
1,1,9197


In [445]:
#pd.value_counts(merged.ServiceDistance).to_frame().reset_index().rename(columns={'index':"service_distance","ServiceDistance":"counts"})

In [446]:

partial_derated=merged.loc[((merged["spn"]==1569) & (merged["fmi"]==31)),:]
partial_derated.shape

(8686, 453)

In [447]:
# warning_derated=warning_derated.drop_duplicates(subset=['EquipmentID', 'EventTimeStamp'], keep="first")
# warning_derated.shape

In [448]:
full_derated=merged.loc[((merged["spn"]==5246) & (merged["fmi"]==0)),:]

In [449]:
full_derated.shape

(511, 453)

In [450]:
full_derated=full_derated.drop_duplicates(subset=['EquipmentID', 'EventTimeStamp'], keep="first")
full_derated.shape

(509, 453)

In [451]:
full_derated.head(5)

,FaultId,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,day,month,Year,winter,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,full_derated,partial_derated,fmi_spn,c_0_101,c_0_110,c_0_125,c_0_157,c_0_175,c_0_190,c_0_32000,c_0_3242,c_0_3246,c_0_3251,c_0_4360,c_0_4363,c_0_5246,c_0_5862,c_10_102,c_10_1761,c_10_3216,c_10_3226,c_10_3251,c_10_3364,c_10_3515,c_10_4360,c_10_4363,c_10_5024,c_10_5031,c_10_5853,c_10_84,c_11_1322,c_11_1326,c_11_1328,c_11_1761,c_11_190,c_11_3360,c_11_3364,c_11_5743,c_11_641,c_12_3226,c_12_3360,c_12_3361,c_12_5742,c_12_5743,c_12_5848,c_12_629,c_12_630,c_12_641,c_13_237,c_13_2791,c_13_3360,c_13_5848,c_13_639,c_13_641,c_13_6713,c_14_3936,c_14_4792,c_14_521032,c_14_641,c_15_101,c_15_102,c_15_157,c_15_3242,c_15_3246,c_15_3720,c_15_3936,c_15_95,c_15_97,c_16_101,c_16_102,c_16_1209,c_16_1239,c_16_157,c_16_168,c_16_175,c_16_188,c_16_2659,c_16_3058,c_16_3216,c_16_3242,c_16_3246,c_16_3251,c_16_3327,c_16_3363,c_16_3936,c_16_412,c_16_4331,c_16_4334,c_16_4360,c_16_4363,c_16_444,c_16_4765,c_16_5246,c_16_5742,c_16_5862,c_16_6773,c_16_81,c_16_95,c_16_97,c_17_102,c_17_111,c_17_168,c_17_1761,c_17_3480,c_17_5298,c_17_5394,c_17_65535,c_18_100,c_18_102,c_18_103,c_18_105,c_18_110,c_18_111,c_18_1127,c_18_1176,c_18_157,c_18_167,c_18_168,c_18_1761,c_18_188,c_18_3031,c_18_3058,c_18_3060,c_18_3364,c_18_3521,c_18_3556,c_18_3597,c_18_4094,c_18_4331,c_18_4334,c_18_4364,c_18_444,c_18_4607,c_18_5298,c_18_5585,c_18_5851,c_18_65535,c_19_3360,c_19_84,c_19_91,c_1_100,c_1_111,c_1_1176,c_1_1761,c_1_3556,c_1_4334,c_20_102,c_20_3216,c_20_3226,c_20_65535,c_21_3226,c_21_5835,c_29_196608,c_2_100,c_2_101,c_2_102,c_2_105,c_2_110,c_2_1172,c_2_1209,c_2_1279,c_2_171,c_2_175,c_2_190,c_2_255,c_2_2629,c_2_27,c_2_3031,c_2_3216,c_2_3218,c_2_3226,c_2_3228,c_2_3241,c_2_3245,c_2_3246,c_2_3251,c_2_3360,c_2_3361,c_2_3480,c_2_3482,c_2_3556,c_2_3610,c_2_3695,c_2_411,c_2_4334,c_2_4363,c_2_4375,c_2_4765,c_2_5019,c_2_51,c_2_5569,c_2_5745,c_2_5851,c_2_612,c_2_65535,c_2_723,c_2_7854,c_2_84,c_2_91,c_31_102,c_31_110,c_31_1322,c_31_1323,c_31_1324,c_31_1325,c_31_1326,c_31_1327,c_31_1328,c_31_1569,c_31_3362,c_31_3703,c_31_4094,c_31_4096,c_31_4364,c_31_4794,c_31_4796,c_31_524287,c_31_5319,c_31_5321,c_31_5397,c_31_633,c_31_641,c_31_65535,c_31_6802,c_31_7827,c_3_100,c_3_101,c_3_102,c_3_105,c_3_1072,c_3_1073,c_3_1075,c_3_108,c_3_110,c_3_111,c_3_1172,c_3_1209,c_3_1347,c_3_1349,c_3_157,c_3_171,c_3_175,c_3_1761,c_3_2623,c_3_3031,c_3_3242,c_3_3246,c_3_3251,c_3_3361,c_3_3363,c_3_3364,c_3_3464,c_3_3480,c_3_3482,c_3_3490,c_3_3513,c_3_3610,c_3_411,c_3_412,c_3_4334,c_3_4340,c_3_4342,c_3_4344,c_3_4360,c_3_4363,c_3_4376,c_3_4765,c_3_4813,c_3_5019,c_3_5097,c_3_51,c_3_5394,c_3_5491,c_3_5742,c_3_5743,c_3_5835,c_3_5862,c_3_647,c_3_65535,c_3_677,c_3_6780,c_3_729,c_3_7854,c_3_862,c_3_96,c_3_97,c_4_100,c_4_101,c_4_102,c_4_108,c_4_110,c_4_111,c_4_1172,c_4_1209,c_4_1347,c_4_171,c_4_1761,c_4_2623,c_4_27,c_4_3031,c_4_3216,c_4_3226,c_4_3242,c_4_3245,c_4_3246,c_4_3251,c_4_3361,c_4_3363,c_4_3480,c_4_3490,c_4_3509,c_4_3510,c_4_3511,c_4_3512,c_4_3513,c_4_3583,c_4_3610,c_4_411,c_4_4334,c_4_4340,c_4_4342,c_4_4360,c_4_4363,c_4_4375,c_4_4376,c_4_4765,c_4_5019,c_4_520953,c_4_5394,c_4_5491,c_4_5742,c_4_5743,c_4_5745,c_4_5746,c_4_5835,c_4_5848,c_4_5862,c_4_7321,c_4_7854,c_4_91,c_4_97,c_5_2791,c_5_3361,c_5_3363,c_5_3556,c_5_4340,c_5_4342,c_5_4344,c_5_4346,c_5_4376,c_5_4607,c_5_5394,c_5_5491,c_5_651,c_5_652,c_5_653,c_5_654,c_5_655,c_5_65535,c_5_656,c_6_2791,c_7_103,c_7_3362,c_7_3363,c_7_3464,c_7_3482,c_7_3490,c_7_3936,c_7_4339,c_

In [452]:
# 193 Aldi Blvd, Mt Juliet, TN – 36.0666667, -86.4347222

# 137 Eady Road – Bell Buckle, TN – 35.5883333, -86.4438888

# 5754 Fish Hatchery Road – Russellville, TN  36.1950, -83.174722

# ["Latitude","Longitude"]

full_derated=full_derated.loc[~((full_derated["Latitude"]==36.0) & (full_derated["Longitude"]==-86.4)),:]
full_derated=full_derated.loc[~((full_derated["Latitude"]==35.5) & (full_derated["Longitude"]==-86.4)),:]
full_derated=full_derated.loc[~((full_derated["Latitude"]==36.1) & (full_derated["Longitude"]==-83.1)),:]

In [453]:
full_derated["ecuMake"].value_counts()

CMMNS    509
Name: ecuMake, dtype: int64

In [454]:
full_derated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 509 entries, 18 to 409386
Columns: 453 entries, FaultId to c_9_96
dtypes: bool(1), datetime64[ns](1), float64(417), int32(1), int64(13), object(20)
memory usage: 1.8+ MB


In [455]:
bad_truck=full_derated['EquipmentID'].unique().tolist()

In [456]:
bad_truck.head()


[1395,
 1630,
 1419,
 1486,
 1492,
 1383,
 1396,
 1602,
 1431,
 1795,
 1595,
 1366,
 1399,
 1417,
 1591,
 1490,
 1696,
 1437,
 1584,
 1692,
 1590,
 1398,
 1418,
 1472,
 1560,
 1375,
 1566,
 1778,
 1659,
 1403,
 1567,
 1373,
 1551,
 1571,
 1443,
 1378,
 1452,
 1488,
 1384,
 1582,
 1501,
 1407,
 1603,
 1477,
 1389,
 1519,
 1757,
 1458,
 1401,
 1467,
 1579,
 1575,
 1601,
 1657,
 1592,
 1563,
 1557,
 1605,
 1581,
 1811,
 1573,
 1594,
 1668,
 1643,
 1561,
 1940,
 1839,
 1552,
 1637,
 1554,
 1686,
 1598,
 1828,
 1848,
 1572,
 1831,
 1661,
 1654,
 2011,
 1917,
 1812,
 1803,
 1814,
 1556,
 1564,
 1585,
 1663,
 1623,
 1800,
 1732,
 1818,
 2009,
 1946,
 1559,
 1711,
 1600,
 1939,
 1704,
 1808,
 1743,
 1822,
 1739,
 1764,
 1846,
 2007,
 1669,
 1819,
 1780,
 2143,
 1718,
 2111,
 1874,
 1689,
 1932,
 1731,
 1794,
 1978,
 1691,
 1942,
 1928,
 1885,
 1921,
 1860,
 1877,
 1878,
 306,
 1853,
 1758,
 1919,
 1958,
 1683,
 1847,
 2209,
 1815,
 2005,
 2082,
 1871,
 1857,
 1698,
 1827,
 1918,
 1751,
 1961,


In [457]:

bad_ones=faults.loc[faults['EquipmentID'].isin(bad_truck),:]
bad_ones.shape

(186558, 24)

In [458]:
bad_ones.head()

,FaultId,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,day,month,Year,winter
5,6,990431,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000,21,2,2015,1
9,10,990446,2015-02-21 11:41:55,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1417,105438630,33.039953,-96.182592,2015-02-21 11:41:51.000,21,2,2015,1
10,11,990459,2015-02-21 11:42:19,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1582,105410980,40.158055,-80.151018,2015-02-21 11:42:55.000,21,2,2015,1
13,14,990470,2015-02-21 11:43:40,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1582,105410980,40.161944,-80.161018,2015-02-21 11:43:36.000,21,2,2015,1
15,16,990495,2015-02-21 11:45:06,Low (Severity Low) Engine Coolant Level,NaN,04993120*00190002*082113134117*07700053*I0*BBZ*,79623056,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1601,105430420,38.272037,-78.937129,2015-02-21 11:45:02.000,21,2,2015,1


In [459]:
bok=bad_ones.groupby(["EquipmentID"])["EquipmentID"].agg({np.size})
bok=bok.sort_values(["size"])
bok.head()

,size
EquipmentID,
2209,15
2009,22
2082,30
1827,33
2111,38
1940,56
1978,62
1477,62
1492,64


In [460]:
bok['size'].mean()

1295.5416666666667

In [461]:
good_ones=faults.loc[~(faults['EquipmentID'].isin(bad_truck)),:]
good_ones.shape

(223201, 24)

In [462]:
iyi=good_ones.groupby(["EquipmentID"])["EquipmentID"].agg({np.size})
iyi=iyi.sort_values(["size"])
iyi.head()

,size
EquipmentID,
2197,1
105355995,1
105340140,1
105363528,1
105337895,1
105322231,1
105311862,1
105311799,1
1672,1


In [463]:
iyi['size'].mean()

309.1426592797784

In [464]:
bk=full_derated.groupby(["Latitude","Longitude"])["Latitude","Longitude"].agg({np.size})
bk=bk["Latitude"].sort_values(["size"])
bk.tail(50)

,,size
Latitude,Longitude,
35.491203,-86.429583,1.0
35.490787,-86.410277,1.0
36.023888,-86.605694,1.0
36.023101,-87.341898,1.0
36.023055,-87.343750,1.0
36.022824,-87.341990,1.0
35.393981,-85.991944,1.0
35.435046,-80.667546,1.0
36.022685,-86.605972,1.0


In [465]:
service = pd.read_excel('data\\Service Fault Codes_1_0_0_167.xlsx')

In [466]:
service.head()

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...


In [467]:

service.groupby(["SPN","J1939 FMI"]).size().reset_index()

,SPN,J1939 FMI,0
0,22,3,1
1,22,4,1
2,27,2,1
3,27,3,1
4,27,4,1
5,27,13,1
6,27,16,1
7,29,2,1
8,38,2,1
9,38,3,1


In [468]:
service["SPN"].value_counts()

524286    765
524287    585
611        86
3464       27
102        22
1761       21
105        21
110        21
7581       21
633        21
641        19
1172       18
190        18
3226       18
3216       18
1127       17
1244       17
3364       17
91         17
157        16
8289       16
2791       16
3465       16
159        15
94         15
629        15
103        15
1075       15
3031       15
3470       15
3251       15
1209       14
84         14
3361       14
1292       14
1176       14
51         14
3227       14
3217       14
4363       14
175        14
1188       13
111        13
632        13
4360       13
4334       13
639        13
4765       13
171        12
3246       12
100        12
1231       11
649        11
108        11
97         11
5848       11
1390       11
3057       11
724        11
5401       11
3242       11
5749       11
3249       11
8322       11
168        10
1300       10
132        10
1301       10
521209     10
1131       10
3265       10
1325  

In [469]:
service["Lamp Color"].value_counts()

Amber          4971
Red             881
None            871
Maintenance     401
Name: Lamp Color, dtype: int64

In [470]:
service["J1939 FMI"].value_counts()

31    2331
3      754
4      730
2      426
16     372
18     267
0      233
15     216
9      211
11     191
5      182
7      179
14     143
17     138
13     133
1      120
12     108
10     104
6       86
19      77
20      61
21      49
8       13
Name: J1939 FMI, dtype: int64

In [471]:
output = service.pivot_table(index='SPN', columns='J1939 FMI', values='Cummins Fault Code')

In [472]:
output.head(5)

J1939 FMI,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,31
SPN,,,,,,,,,,,,,,,,,,,,,,,
22,NaN,NaN,NaN,719.0,729.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,1228.0,2271.0,2272.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2348.0,NaN,NaN,1625.0,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,149.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,NaN,NaN,5319.0,5317.0,5318.0,NaN,NaN,NaN,NaN,6342.0,NaN,NaN,NaN,6343.0,NaN,NaN,NaN,NaN,1439.0,5328.0,NaN,NaN,NaN
46,NaN,NaN,218.0,216.0,217.0,774.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,775.0,776.0,NaN,NaN,NaN,NaN


In [473]:
# lat=38
# long=-84

# map=folium.Map(location=[lat,long],zoom_start=2)

# map2=folium.Map(location=[lat,long],zoom_start=2)
# map3=folium.Map(location=[lat,long],zoom_start=2)

In [474]:
# for row in full_derated.iterrows():
#     row_values = row[1]
#     location = [row_values['Latitude'], row_values['Longitude']]
#     marker = folium.CircleMarker(location=location,radius=1)
#     marker.add_to(map)
# map

In [475]:
truck_no_idea=[105360462,105411041,105427203,105349576]

In [476]:
truck_no_idea=faults.loc[(faults['MCTNumber'].isin(truck_no_idea)),:]
truck_no_idea.shape

(7096, 24)

In [477]:
truck_no_idea

,FaultId,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,day,month,Year,winter
2059,2060,1010491,2015-02-23 03:59:52,High (Severity Low) Aftertreatment Diesel Part...,NaN,04993120*00028586*040213150018*07700044*I0*BBZ*,79464956,6X1u10D1500000000,CMMNS,0,3936,15,False,1,NaN,1386,105349576,35.713703,-85.723703,2015-02-23 04:00:42.000,23,2,2015,1
2060,2061,1010492,2015-02-23 03:59:59,High (Severity Medium) Engine Exhaust Gas Reci...,NaN,04993120*00028586*040213150018*07700044*I0*BBZ*,79464956,6X1u10D1500000000,CMMNS,0,3058,16,True,1,NaN,1386,105349576,35.713703,-85.723703,2015-02-23 04:00:35.000,23,2,2015,1
2550,2551,1018783,2015-02-23 11:04:35,High (Severity Medium) Engine Exhaust Gas Reci...,NaN,04993120*00028586*040213150018*07700044*I0*BBZ*,79464956,6X1u10D1500000000,CMMNS,0,3058,16,False,1,NaN,1386,105349576,32.361481,-84.961944,2015-02-23 11:04:31.000,23,2,2015,1
3811,3812,1042518,2015-02-24 09:26:14,Low (Severity Low) Engine Coolant Level,NaN,04993120*00174945*060712145341*07700027*I0*BBZ*,79621367,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1598,105411041,38.186990,-84.803379,2015-02-24 09:13:18.000,24,2,2015,1
3812,3813,1042537,2015-02-24 09:26:38,Low (Severity Low) Engine Coolant Level,NaN,04993120*00174945*060712145341*07700027*I0*BBZ*,79621367,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1598,105411041,38.180462,-84.805648,2015-02-24 09:27:14.000,24,2,2015,1
5742,5743,1070943,2015-02-25 14:07:04,Low (Severity Low) Engine Coolant Level,NaN,04993120*00174945*060712145341*07700027*I0*BBZ*,79621367,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1598,105411041,32.518287,-90.093564,2015-02-25 14:06:59.000,25,2,2015,1
5744,5745,1070967,2015-02-25 14:07:35,Low (Severity Low) Engine Coolant Level,NaN,04993120*00174945*060712145341*07700027*I0*BBZ*,79621367,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1598,105411041,32.518055,-90.093935,2015-02-25 14:08:10.000,25,2,2015,1
5806,5807,1071852,2015-02-25 14:43:53,NaN,NaN,04993120*00028586*040213150018*07700044*I0*BBZ*,79464956,6X1u10D1500000000,CMMNS,0,5397,31,True,1,NaN,1386,105349576,36.829675,-81.503379,2015-02-25 14:44:30.000,25,2,2015,1
5951,5952,1073690,2015-02-25 16:08:21,Low (Severity Low) Engine Coolant Level,NaN,04993120*00174945*060712145341*07700027*I0*BBZ*,79621367,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1598,105411041,31.825324,-89.398240,2015-02-25 16:08:16.000,25,2,2015,1
5954,5955,1073740,2015-02-25 16:09:21,Low (Severity Low) Engine Coolant Level,NaN,04993120*00174945*060712145341*07700027*I0*BBZ*,79621367,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1598,105411041,31.826805,-89.399768,2015-02-25 16:09:57.000,25,2,2015,1


In [478]:
pd.crosstab(truck_no_idea["EquipmentID"], truck_no_idea["MCTNumber"], dropna=True)

MCTNumber,105349576,105360462,105411041
EquipmentID,,,
1386,4,0,0
1576,86,0,0
1598,0,0,508
1696,0,412,0
1809,0,0,6086


In [479]:
cols = [c for c in regression_data.columns if c.lower()[:2] == 'c_']
cols=cols+["BarometricPressure",             
"CruiseControlSetSpeed",          
"DistanceLtd",                    
"EngineCoolantTemperature",       
"EngineLoad",                     
"EngineOilPressure",              
"EngineOilTemperature",           
"EngineRpm",                      
"EngineTimeLtd",                  
"Speed",                          
"TurboBoostPressure", 
"winter",                                 
"full_derated",
]
regression_data=regression_data[cols]

In [481]:
X = pd.DataFrame(regression_data)
y = pd.Series("full_derated")
X = X.fillna(X.median())

In [482]:
#imputer= Imputer(strategy="most_frequent")

#imputer.fit_transform(X)

In [483]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 409759 entries, 0 to 409758
Columns: 415 entries, c_0_101 to full_derated
dtypes: float64(413), int64(2)
memory usage: 1.3 GB


In [484]:
X.head()

,c_0_101,c_0_110,c_0_125,c_0_157,c_0_175,c_0_190,c_0_32000,c_0_3242,c_0_3246,c_0_3251,c_0_4360,c_0_4363,c_0_5246,c_0_5862,c_10_102,c_10_1761,c_10_3216,c_10_3226,c_10_3251,c_10_3364,c_10_3515,c_10_4360,c_10_4363,c_10_5024,c_10_5031,c_10_5853,c_10_84,c_11_1322,c_11_1326,c_11_1328,c_11_1761,c_11_190,c_11_3360,c_11_3364,c_11_5743,c_11_641,c_12_3226,c_12_3360,c_12_3361,c_12_5742,c_12_5743,c_12_5848,c_12_629,c_12_630,c_12_641,c_13_237,c_13_2791,c_13_3360,c_13_5848,c_13_639,c_13_641,c_13_6713,c_14_3936,c_14_4792,c_14_521032,c_14_641,c_15_101,c_15_102,c_15_157,c_15_3242,c_15_3246,c_15_3720,c_15_3936,c_15_95,c_15_97,c_16_101,c_16_102,c_16_1209,c_16_1239,c_16_157,c_16_168,c_16_175,c_16_188,c_16_2659,c_16_3058,c_16_3216,c_16_3242,c_16_3246,c_16_3251,c_16_3327,c_16_3363,c_16_3936,c_16_412,c_16_4331,c_16_4334,c_16_4360,c_16_4363,c_16_444,c_16_4765,c_16_5246,c_16_5742,c_16_5862,c_16_6773,c_16_81,c_16_95,c_16_97,c_17_102,c_17_111,c_17_168,c_17_1761,c_17_3480,c_17_5298,c_17_5394,c_17_65535,c_18_100,c_18_102,c_18_103,c_18_105,c_18_110,c_18_111,c_18_1127,c_18_1176,c_18_157,c_18_167,c_18_168,c_18_1761,c_18_188,c_18_3031,c_18_3058,c_18_3060,c_18_3364,c_18_3521,c_18_3556,c_18_3597,c_18_4094,c_18_4331,c_18_4334,c_18_4364,c_18_444,c_18_4607,c_18_5298,c_18_5585,c_18_5851,c_18_65535,c_19_3360,c_19_84,c_19_91,c_1_100,c_1_111,c_1_1176,c_1_1761,c_1_3556,c_1_4334,c_20_102,c_20_3216,c_20_3226,c_20_65535,c_21_3226,c_21_5835,c_29_196608,c_2_100,c_2_101,c_2_102,c_2_105,c_2_110,c_2_1172,c_2_1209,c_2_1279,c_2_171,c_2_175,c_2_190,c_2_255,c_2_2629,c_2_27,c_2_3031,c_2_3216,c_2_3218,c_2_3226,c_2_3228,c_2_3241,c_2_3245,c_2_3246,c_2_3251,c_2_3360,c_2_3361,c_2_3480,c_2_3482,c_2_3556,c_2_3610,c_2_3695,c_2_411,c_2_4334,c_2_4363,c_2_4375,c_2_4765,c_2_5019,c_2_51,c_2_5569,c_2_5745,c_2_5851,c_2_612,c_2_65535,c_2_723,c_2_7854,c_2_84,c_2_91,c_31_102,c_31_110,c_31_1322,c_31_1323,c_31_1324,c_31_1325,c_31_1326,c_31_1327,c_31_1328,c_31_1569,c_31_3362,c_31_3703,c_31_4094,c_31_4096,c_31_4364,c_31_4794,c_31_4796,c_31_524287,c_31_5319,c_31_5321,c_31_5397,c_31_633,c_31_641,c_31_65535,c_31_6802,c_31_7827,c_3_100,c_3_101,c_3_102,c_3_105,c_3_1072,c_3_1073,c_3_1075,c_3_108,c_3_110,c_3_111,c_3_1172,c_3_1209,c_3_1347,c_3_1349,c_3_157,c_3_171,c_3_175,c_3_1761,c_3_2623,c_3_3031,c_3_3242,c_3_3246,c_3_3251,c_3_3361,c_3_3363,c_3_3364,c_3_3464,c_3_3480,c_3_3482,c_3_3490,c_3_3513,c_3_3610,c_3_411,c_3_412,c_3_4334,c_3_4340,c_3_4342,c_3_4344,c_3_4360,c_3_4363,c_3_4376,c_3_4765,c_3_4813,c_3_5019,c_3_5097,c_3_51,c_3_5394,c_3_5491,c_3_5742,c_3_5743,c_3_5835,c_3_5862,c_3_647,c_3_65535,c_3_677,c_3_6780,c_3_729,c_3_7854,c_3_862,c_3_96,c_3_97,c_4_100,c_4_101,c_4_102,c_4_108,c_4_110,c_4_111,c_4_1172,c_4_1209,c_4_1347,c_4_171,c_4_1761,c_4_2623,c_4_27,c_4_3031,c_4_3216,c_4_3226,c_4_3242,c_4_3245,c_4_3246,c_4_3251,c_4_3361,c_4_3363,c_4_3480,c_4_3490,c_4_3509,c_4_3510,c_4_3511,c_4_3512,c_4_3513,c_4_3583,c_4_3610,c_4_411,c_4_4334,c_4_4340,c_4_4342,c_4_4360,c_4_4363,c_4_4375,c_4_4376,c_4_4765,c_4_5019,c_4_520953,c_4_5394,c_4_5491,c_4_5742,c_4_5743,c_4_5745,c_4_5746,c_4_5835,c_4_5848,c_4_5862,c_4_7321,c_4_7854,c_4_91,c_4_97,c_5_2791,c_5_3361,c_5_3363,c_5_3556,c_5_4340,c_5_4342,c_5_4344,c_5_4346,c_5_4376,c_5_4607,c_5_5394,c_5_5491,c_5_651,c_5_652,c_5_653,c_5_654,c_5_655,c_5_65535,c_5_656,c_6_2791,c_7_103,c_7_3362,c_7_3363,c_7_3464,c_7_3482,c_7_3490,c_7_3936,c_7_4339,c_7_4376,c_7_4752,c_7_5394,c_7_5491,c_7_641,c_7_651,c_7_652,c_7_653,c_7_654,c_7_655,c_7_656,c_7_723,c_8_2623,c_8_91,c_9_1081,c_9_1761,c_9_191,c_9_3031,c_9_3216,c_9_3226,c_9_3360,c_9_3364,c_9_38,c_9_558,c_9_563,c_9_5742,c_9_5743,c_9_5835,c_9_5848,c_9_639,c_9_641,c_9_65535,c_9_6713,c_9_84,c_9_91,c_9_96,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,Speed,TurboBoostPressure,winter,full_derated
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0

In [485]:
X_train, X_test, y_train, y_test = train_test_split(X, X.full_derated, test_size = 0.5, random_state=42)

In [486]:
lg_model = LogisticRegression()

lg_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [487]:
accuracy_score(y_test, lg_model.predict(X_test))

0.977328192112456

In [488]:
confusion_matrix(y_test, lg_model.predict(X_test))

array([[200235,      0],
       [  4645,      0]], dtype=int64)

In [489]:

roc_auc_score(y_test, lg_model.predict(X_test))

0.5